In [18]:
%run functions/cytoscapeFunctions

In [19]:
from py2cytoscape import cyrest
import networkx as nx
import numpy as np
import pandas as pd
import random
import time
import tempfile
from networkx.generators.random_graphs import barabasi_albert_graph as ba

In [20]:
G = nx.fast_gnp_random_graph(10, 0.4)
G1 = nx.fast_gnp_random_graph(10, 0.6)

In [21]:
def addEdgeAttrib (nxGraph, attribName, lowNum, highNum):
    edges = nxGraph.edges(data = True)
    
    for i in range (len(edges)):
        nxGraph[list(edges)[i][0]][list(edges)[i][1]][attribName] = float(random.randint(lowNum, highNum))

In [22]:
def addHalfEdgeAttrib (nxGraph, attribName, lowNum1, highNum1, lowNum2, highNum2):
    edges = nxGraph.edges(data = True)
    half = edges/2
    
    for i in range (half):
        nxGraph[list(edges)[i][0]][list(edges)[i][1]][attribName] = random.randint(lowNum1, highNum1)
        
    for i in range(half):
        nxGraph[list(edges)[i+(half)][0]][list(half)[i][1]][attribName] = random.randint(lowNum2, highNum2)

In [142]:
addEdgeAttrib(G, 'weight', 0,0)
# addEdgeAttrib(G, 'test',1,10)
addEdgeAttrib(G1, 'weight',1,10)
# addEdgeAttrib(G1, 'test',1,10)

In [24]:
# if True, return True and element in inputList
def contains (element, inputList):
    for i in range (len(inputList)):
        if inputList[i]==element:
            return (True, i)
    return False

In [25]:
# preprossesing
# assuming some attributes exist
def conformEdges (network1, network2):
    edges1 = list(network1.edges())
    edges2 = list(network2.edges())
    
    keys = list(list(network1.edges(data = True))[0][2].keys())

    # ranging thru edges1 and checking if
    for i in range (len(edges1)):
        if (not(contains(edges1[i], edges2))):
            network2.add_edge(edges1[i][0],edges1[i][1])
            for key in keys:
                network2[edges1[i][0]][edges1[i][1]][key] = 0

    for i in range(len(edges2)):
        if (not(contains(edges2[i], edges1))):
            network1.add_edge(edges2[i][0], edges2[i][1])
            for key in keys:
                network1[edges2[i][0]][edges2[i][1]][key] = 0

In [27]:
# edges1 and edges2 should have the same length, going from edge1 -> edge2
def genShifts(edges1, edges2, columnName, numNetworks):
    numEdges = len(edges1)
    shifts = []
    
    for i in range(numEdges):
        a = list(edges1)[i][2][columnName]
        b = list(edges2)[i][2][columnName]
        shifts.append((b-a)/(numNetworks-1))
    
    return shifts

In [135]:
# assume edgeLists same length, and same attribute catagories
def fillBetweenEdges(nxGraph1, nxGraph2, numNetworks):
    edges1, edges2 = list(nxGraph1.edges(data = True)), list(nxGraph2.edges(data = True))
    numEdges = len(edges1)
    keys = list(list(edges1)[0][2].keys())
    networks = []
    
    for networkNum in range(numNetworks):
        tempNetwork = nxGraph1.copy()
        for attrib in keys:
            for edge in range(numEdges):
                source = edges1[edge][0]
                target = edges1[edge][1]
                val1, val2 = nxGraph1[source][target][attrib], nxGraph2[source][target][attrib]
                
                tempNetwork[source][target][attrib] = np.linspace(val1, val2, numNetworks)[networkNum]
                
        networks.append(tempNetwork)
        
    return networks

In [152]:
networks = fillBetweenEdges(G, G1, 3)

In [153]:
convertEdge(networks[1])

,source,target,weight
0,0,2,4.5
1,0,4,2.5
2,0,7,4.0
3,0,9,5.0
4,0,3,2.5
5,0,5,1.0
6,0,6,4.5
7,1,5,3.5
8,1,8,1.0
9,1,4,3.0


In [147]:
convertEdge(G1)

,source,target,weight
0,0,2,9.0
1,0,3,5.0
2,0,4,8.0
3,0,5,10.0
4,0,6,5.0
5,0,7,2.0
6,0,9,9.0
7,1,4,7.0
8,1,6,2.0
9,1,7,6.0


In [ ]:
# postprocessing after network list has already been made
def deleteZeroWeights (networks):
    for network in networks:
        edges = list(network.edges(data = True))
        keys = list(edges[0][2].keys())

        for edge in edges:
            empty = True
            for key in keys:
                if network[edge[0]][edge[1]][key] > 0:
                    empty = False
            if empty == True:
                network.remove_edge(edge[0], edge[1])

In [ ]:
# path = os.path.join(os.path.dirname(os.path.abspath("template.cys")), "sample.csv")
# for i in range(len(networks)):
#     convertEdge(networks[i]).to_csv(path)
#     time.sleep(0.3)